In [ ]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 86.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.linalg
import qiskit as qk
import qiskit.visualization

from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector

from qiskit_aer import AerSimulator
from qiskit_aer.noise import QuantumError, ReadoutError, NoiseModel,depolarizing_error,amplitude_damping_error,kraus_error,pauli_error
from qiskit import transpile
from qiskit.quantum_info.operators import Operator

import tensorflow as tf
from tensorflow.python.framework.ops import convert_to_tensor
print("TensorFlow version:", tf.__version__)

import keras
from keras.layers import Dense, Input, Lambda, concatenate, Conv2D, UpSampling2D, MaxPooling2D, Add,Concatenate,Flatten, Conv1D, Conv1DTranspose, UpSampling1D,MaxPooling1D, BatchNormalization, Dropout
from tensorflow.keras import Model

TensorFlow version: 2.17.0


In [ ]:
# Ensure reproducibility
np.random.seed(123)
tf.random.set_seed(123)
import random
random.seed(10)

In [ ]:
# Generation of a random unitary transformation
def random_unitary(N):
    Z=np.random.randn(N,N) + 1.0j * np.random.randn(N,N)
    [Q,R]=sp.linalg.qr(Z)
    D=np.diag(np.diagonal(R)/np.abs(np.diagonal(R)))
    return np.dot(Q,D)

In [ ]:
# Generate the density matrix given the state vector
def get_density_matrix(state_vector):
    density_matrix = np.outer(state_vector, np.conjugate(state_vector))
    return density_matrix

In [ ]:
# Here we define the identity matrix and the Pauli matrices for dimension 2 (one qubit)
I = np.array([[1, 0],[0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [ ]:
sim_bknd=AerSimulator()

In [ ]:
# Generate Haar distributed data

def generate_Haar_data(num_qubits, samples=1000):
    data = []
    for i in range(samples):
        qc = qk.QuantumCircuit(num_qubits) #creates a quantum circuit with "num_qubits" qubits
        u = random_unitary(2**num_qubits)
        qc.unitary(u, qubits=range(num_qubits)) #applies the random unitary transformation to the circuit
        qc = qk.transpile(qc, backend=sim_bknd) #it's used to optimize the circuit
        qc.save_statevector() #it's the instruction to save the state vector obtained by the simulation

        state = sim_bknd.run(qc).result().get_statevector(qc) #does the simulation and gets the state vector
        state = np.asarray(state)
        data.append(state)
    return data

In [ ]:
# Generate Pauli basis for three-qubit states

pauli_basis1q = np.array([I, X, Y, Z])
pauli_basis2q = np.array([np.kron(a,b) for a in pauli_basis1q for b in pauli_basis1q])

pauli_basis3q = np.array([np.kron(a,b) for a in pauli_basis2q for b in pauli_basis1q])

pauli_basis4q = np.array([np.kron(a,b) for a in pauli_basis3q for b in pauli_basis1q])
pauli_basis4q_modified = pauli_basis4q[1:]

# Compute Bloch components from density matrix
def bloch_coeffs(rho):
    c = []
    for p in pauli_basis4q_modified:
        c.append(np.trace(rho @ p).real)
    return np.array(c)

In [ ]:
# 8x8 Identity matrix
I_8 = tf.eye(16, dtype=tf.complex64)

# Compute fidelity between generalized Bloch vectors
def fid_fourq(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk', pauli_basis4q_modified, a)
   el_b = tf.einsum('ijk,mi->mjk', pauli_basis4q_modified, b)
   rho_a = (1/16) *(el_a + I_8)
   rho_b = (1/16) * (el_b +I_8)
   fidelity = tf.linalg.trace(rho_a @ rho_b)
   return fidelity

In [ ]:
# Define Infidelity as loss function
@tf.function
def infidelity4(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk', pauli_basis4q_modified, a)
   el_b = tf.einsum('ijk,mi->mjk', pauli_basis4q_modified, b)
   rho_a = (1/16) *(el_a + I_8)
   rho_b = (1/16) * (el_b +I_8)
   fidelity = tf.linalg.trace(rho_a @ rho_b)
   infidelity = 1 - fidelity
   infidelity = tf.cast(infidelity, dtype = tf.float32)
   return infidelity

# **<font color='green'> Noise Models</font>**

In [ ]:
# Probabilities for bit, phase, bit-phase flip or depolarizing channels
p_error = 0.2
num_qubits = 1 # for depolarizing channels

# Probabilities for generalized Pauli channel
p_0 = 0.7
p_1 = 0.2
p_2 = 0.05
p_3 = 0.05

# Channels
bit_flip_error = pauli_error([('X', p_error), ('I', 1 - p_error)])
phase_flip_error = pauli_error([('Z', p_error), ('I', 1 - p_error)])
bit_phase_flip_error = pauli_error([('Y', p_error), ('I', 1 - p_error)])
dep_error = depolarizing_error(p_error, num_qubits)
general_pauli_error = pauli_error([('X', p_1), ('Y', p_2), ('Z', p_3), ('I', p_0)])
AD_error = amplitude_damping_error (0.3, 0.5)

In [ ]:
# Apply noise whenever an identity gate appears in the circuit (compatibly with the probability errors)
def qsk_add_error(error_type):
    if error_type == 'bit':
        noise_model.add_all_qubit_quantum_error(bit_flip_error, ["id"])
    if error_type == 'phase':
        noise_model.add_all_qubit_quantum_error(phase_flip_error, ["id"])
    if error_type == 'bit-phase':
        noise_model.add_all_qubit_quantum_error(bit_phase_flip_error, ["id"])
    if error_type == 'depolarizing':
        noise_model.add_all_qubit_quantum_error(dep_error, ["id"])
    if error_type == 'Gpauli':
        noise_model.add_all_qubit_quantum_error(general_pauli_error, ["id"])
    if error_type == 'AD':
        noise_model.add_all_qubit_quantum_error(AD_error, ["id"])
    if error_type == 'Kraus':
        noise_model.add_all_qubit_quantum_error(k_error, ["id"])

# **<font color='green'> State Reconstruction</font>**

In [ ]:
# Generate data
data = generate_Haar_data(4, 5000)
density_matrix_noise_free = [*map(get_density_matrix, data)]

In [ ]:
num_qubits = 4
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):
    #print(f"Sample: {i+1} / {len(data)}")

    circ = qk.QuantumCircuit(num_qubits)

    circ.initialize(data[i])
    circ = transpile(circ, backend=AerSimulator(method='density_matrix'))
    circ.barrier()
    circ.id(0) # applies identity to the first qubit
    circ.id(1) # applies identity to the second qubit
    circ.id(2) # applies identity to the third qubit
    circ.id(3)
    circ.save_density_matrix()
    #print(circ)
    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0])
    noise_model.add_quantum_error(bit_flip_error, ["id"], [1])
    noise_model.add_quantum_error(bit_phase_flip_error, ["id"], [2])
    noise_model.add_quantum_error(dep_error, ["id"], [3])
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [ ]:
bloch_vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noise-free Bloch vectors
bloch_vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noisy Bloch vectors

In [ ]:
# Generate training, validation and test set

x_train_list = bloch_vectors_with_noise[:4000]
y_train_list = bloch_vectors_noise_free[:4000]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = bloch_vectors_with_noise[4000:4500]
y_val_list = bloch_vectors_noise_free[4000:4500]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = bloch_vectors_with_noise[4500:]
y_test_list = bloch_vectors_noise_free[4500:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [ ]:
xlist1=[]
ylist1=[]
for i in range(5000):
  xlist1.append([tf.math.real(den_mat_with_noise[i]),tf.math.imag(den_mat_with_noise[i])])
  ylist1.append([tf.math.real(den_mat_noise_free[i]),tf.math.imag(den_mat_noise_free[i])])


In [ ]:
x_train_list1 = xlist1[:4000]
y_train_list1 = ylist1[:4000]

x_train1 = tf.transpose(tf.convert_to_tensor(x_train_list1), perm=[0, 2, 3, 1])
y_train1 = tf.transpose(tf.convert_to_tensor(y_train_list1), perm=[0, 2, 3, 1])

x_val_list1 = xlist1[4000:4500]
y_val_list1 = ylist1[4000:4500]

x_val1 = tf.transpose(tf.convert_to_tensor(x_val_list1), perm=[0, 2, 3, 1])
y_val1 = tf.transpose(tf.convert_to_tensor(x_val_list1), perm=[0, 2, 3, 1])

x_test_list1 = xlist1[4500:]
y_test_list1 = ylist1[4500:]

x_test1 = tf.transpose(tf.convert_to_tensor(x_test_list1), perm=[0, 2, 3, 1])
y_test1 = tf.transpose(tf.convert_to_tensor(x_test_list1), perm=[0, 2, 3, 1])

mse training

In [ ]:
# Define Model
mlpmodel = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(255,)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(255),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(15.) * tf.math.l2_normalize(x, axis=1))
  ])
# Compile model
adam_opt = tf.optimizers.Adam(0.0002)
mlpmodel.compile(optimizer=adam_opt,
              loss='mse')
mlpmodel.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │          65,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 255)                 │          65,535 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 255)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 262,655 (1.00 MB)

 Trainable params: 262,655 (1.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train Model
history = mlpmodel.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=100, epochs=800)


Epoch 1/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1162 - val_loss: 0.1097
Epoch 2/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1041 - val_loss: 0.0977
Epoch 3/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0914 - val_loss: 0.0889
Epoch 4/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0824 - val_loss: 0.0819
Epoch 5/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0754 - val_loss: 0.0761
Epoch 6/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0696 - val_loss: 0.0711
Epoch 7/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0648 - val_loss: 0.0668
Epoch 8/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0606 - val_loss: 0.0630
Epoch 9/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0569 - val_loss: 0.0597
Epoch 10/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0537 - val_loss: 0.0566
Epoch 11/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0508 - val_loss: 0.0539
Epoch 12/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - los

In [ ]:
score = mlpmodel.evaluate(x_test,  y_test, verbose=2)

16/16 - 0s - 9ms/step - loss: 0.0018


In [ ]:
# save the model predictions in a tensor
y_prediction = mlpmodel(x_test)

y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
#x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_fourq(y_prediction, y_test)


print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9858923554420471-7.072230995275319e-11j)


Infidelity training

In [ ]:
mlpmodel2 = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(255,)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(255),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(15.) * tf.math.l2_normalize(x, axis=1))
  ])
# Compile model
adam_opt = tf.optimizers.Adam(0.0002)
mlpmodel2.compile(optimizer=adam_opt,
              loss=infidelity4)
mlpmodel2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │          65,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 255)                 │          65,535 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 255)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 262,655 (1.00 MB)

 Trainable params: 262,655 (1.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = mlpmodel2.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=100, epochs=800)

Epoch 1/800


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.9226 - val_loss: 0.8739
Epoch 2/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.8232 - val_loss: 0.7760
Epoch 3/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.7226 - val_loss: 0.7067
Epoch 4/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.6516 - val_loss: 0.6515
Epoch 5/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.5967 - val_loss: 0.6056
Epoch 6/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.5520 - val_loss: 0.5665
Epoch 7/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.5141 - val_loss: 0.5328
Epoch 8/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.4816 - val_loss: 0.5032
Epoch 9/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.4532 - val_loss: 0.4771
Epoch 10/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.4282 - val_loss: 0.4534
Epoch 11/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.4057 - val_loss: 0.4317
Epoch 12/800
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.3

In [ ]:
y_prediction = mlpmodel2(x_test)

y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
#x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_fourq(y_prediction, y_test)


print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9801384210586548+1.2433155893720738e-10j)
